In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [5]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [6]:
from typing import Any, Dict, List

import boto3
import yaml

from core.task_processor import TaskProcessor
from core.task_publisher import TaskPublisher

In [7]:
_sqs = None
_processor = None
_queue_url = None

In [8]:
with open("../config/tenants/dev.yaml", "r", encoding="utf-8") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {}) or {}
dynamodb = boto3.resource("dynamodb", region_name=dynamodb_config.get("region"))

messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))
contacts_table = dynamodb.Table(dynamodb_config.get("contacts_table"))
tasks_table = dynamodb.Table(dynamodb_config.get("tasks_table"))

sqs_config = tenant_config.get("sqs", {}) or {}
_sqs = boto3.client("sqs", region_name=sqs_config.get("region"))

_queue_url = (sqs_config.get("tasks_url"))

_processor = TaskProcessor(
    tenant_config=tenant_config,
    messages_table=messages_table,
    processes_table=processes_table,
    contacts_table=contacts_table,
    tasks_table=tasks_table,
    task_publisher=TaskPublisher(_sqs, _queue_url),
)

print("INIT: processor ready")

INIT: processor ready


In [9]:
import json

body = {
    "task_id": "e7c671d1-92b4-45b6-8a68-01761a1462ed",
    "task_type": "VALIDATE_DOCUMENT",
    "agent_type": "ACCOUNTING_ASSISTANT",
    "process_type": "WHATSAPP_DOCUMENT_PIPELINE",
    "context_key": {
        "msg_id": "wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDRBMDgzMzg3MjQ4MTA5ODU3RkQ1AA==",
        "document_id": "4344525979124995"
    },
    "payload": {},
    "timestamp_iso": "2025-12-18T21:11:58.638486+00:00",
    "timestamp_epoch": 1766092318
}
task = json.dumps(body)

In [10]:
try:
    message_id = "Fake"
    processed, remaining = _processor.process(task)
    print(f"PROCESS: messageId={message_id} processed={processed} remaining={remaining}")

    if remaining < 0:
        print(f"DROP: messageId={message_id} stale")

    delay = max(0, min(int(remaining), 43200))
    print(f"DEFER: messageId={message_id} delay={delay}s")

except Exception as e:
    print(f"ERROR: messageId={message_id} err={repr(e)}")

validate_document
📤 Publishing task b5217e55-2ede-4734-9c00-2d976ffc6cb4 → DETECT_PDF_ENCRYPTION delay=0s
PROCESS: messageId=Fake processed=True remaining=0
DEFER: messageId=Fake delay=0s


In [11]:
processed

True